In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
# Importing the necessary library 'drive' from the package 'google.colab'
from google.colab import drive

# Mounting the Google Drive to the Colab runtime
# This allows access to files and directories in the Google Drive
drive.mount('/content/gdrive')

In [ ]:
#This line of code installs the FiftyOne library using pip.
#FiftyOne is a Python package that provides interactive exploration, analysis, and visualization of datasets for computer vision tasks.
!pip install fiftyone

In [ ]:
import random  # Importing the random module for generating random numbers
import fiftyone as fo  # Importing the fiftyone library for working with datasets and models
import fiftyone.zoo as foz  # Importing the fiftyone.zoo module for accessing pre-trained models
from fiftyone import ViewField as F  # Importing the ViewField class from the fiftyone module

In [ ]:
# Define a list of COCO classes for object relabeling
'''
relabeling_classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train",
    "truck", "boat", "traffic light", "fire hydrant", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep",
    "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot",
    "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant",
    "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
    "hair drier", "toothbrush"
]
relabeling_classes = [
    "bird", "cat", "dog", "horse", "sheep",
    "cow", "elephant", "bear", "zebra", "giraffe"
]

labeling_classes = [
    "bird", "cat", "dog", "horse", "sheep",
    "cow", "elephant", "bear", "zebra", "giraffe"
]
'''
#shrink relabeling classes to 3 for faster training.
relabeling_classes = [
    "cat", "dog", "horse"
]

# this is used to export the class list to the converted yolov4 format data.

labeling_classes = [
    "cat", "dog", "horse"
]

In [ ]:
def dwonload_dataset(class_name, max_samples):

  dataset = foz.load_zoo_dataset( # Load a dataset from the zoo using foz.load_zoo_dataset() function
      "coco-2017",
      splits=["train"],
      classes= class_name,        # Specify the name of the dataset as "coco-2017"
      max_samples= max_samples   # Specify the maximum number of samples to load from the dataset based on the provided max_samples argument
  )
  dataset.persistent = True       # Set the persistent attribute of the dataset to True, indicating that it should be saved and loaded from disk if necessary

  return dataset

In [ ]:
def clone_and_split_test(dataset, class_name):

  class_dataset = dataset.clone()  # Clone the original dataset
  class_dataset.persistent = True  # Set the persistent flag to True, indicating that the cloned dataset should persist in memory

  class_test_dataset = dataset.clone()  # Clone the original dataset
  class_test_dataset.persistent = True  # Set the persistent flag to True, indicating that the cloned dataset should persist in memory

  class_dataset_view = class_dataset.filter_labels("ground_truth", F("label").is_in([class_name])).limit(1000)  # Create a view of the class_dataset, filtered by the given class_name and limited to 1000 samples

  class_test_view = class_test_dataset.filter_labels("ground_truth", F("label").is_in([class_name])).skip(1000)  # Create a view of the class_test_dataset, filtered by the given class_name and skipping the first 1000 samples

  return class_dataset_view.clone(), class_test_view.clone()  # Return the cloned views of the class_dataset and class_test_dataset


In [ ]:
def merge_dataset(dataset, class_name):

  temp1 = dataset.clone()  # Create a clone of the dataset and store it in 'temp1' variable
  temp = temp1.filter_labels("ground_truth", F("label").is_in(["cat"]))  # Filter the dataset to include only samples with the label "cat" in the "ground_truth" field, and store the result in 'temp' variable

  for sample in temp.select_fields('tags').iter_samples(autosave=True):  # Iterate over each sample in 'temp' dataset, selecting only the 'tags' field, and enabling autosave
    sample.set_field('tags', ['train'])  # Set the 'tags' field of the current sample to ['train']

  temp.keep()  # Remove any intermediate data and keep only the modified dataset 'temp'

  return temp.clone()  # Return a clone of the modified dataset 'temp'


In [ ]:
def resize_dataset(dataset, percentage):

  print(len(dataset))  # Print the length of the dataset to check its initial size
  temp = dataset.take(int(len(dataset) * percentage/100))  # Take a portion of the dataset based on the given percentage

  return temp.clone()  # Return a copy of the extracted portion of the dataset


In [ ]:
def resplit(dataset, percentage):

  total_len = len(dataset)  # Calculate the total length of the dataset
  train_len = int(total_len * percentage / 100)  # Calculate the length of the training set based on the percentage
  val_len = total_len - train_len  # Calculate the length of the validation set by subtracting the training set length from the total length

  print('train:', train_len, ' validiation:', val_len, ' total:', total_len)  # Print the lengths of the training set, validation set, and total dataset

  train_set = dataset.limit(train_len)  # Limit the dataset to the length of the training set to create the training set
  val_set = dataset.skip(train_len).limit(val_len)  # Skip the training set length and limit the dataset to the length of the validation set to create the validation set


  for sample in train_set.select_fields('tags').iter_samples(autosave=True):
    sample.set_field('tags', ['train'])  # Set the 'tags' field of each sample in the training set to ['train']

  for sample in val_set.select_fields('tags').iter_samples(autosave=True):
    sample.set_field('tags', ['validation'])  # Set the 'tags' field of each sample in the validation set to ['validation']

  return train_set.clone(), val_set.clone()  # Return the cloned versions of the training set and validation set


In [ ]:
def add_label_error(dataset, percentage, class_name):

  total_len = len(dataset)
  label_error_len = int(total_len * percentage / 100)
  error_free_len = total_len - label_error_len

  # randomly split the dataset into error and error free sets using the given ratio
  label_error_set = dataset.limit(label_error_len)
  error_free_set = dataset.skip(label_error_len).limit(error_free_len)

  # Check if there are three relabeling classes and remove the specified class
  if (len(relabeling_classes)==3):
    relabeling_classes.remove(class_name)

  # Iterate over each sample in the label error set
  for sample in label_error_set.iter_samples(autosave=True):

    # Iterate over each detection in the sample's ground truth
    for detection in sample.ground_truth.detections:
      # Generate a new label randomly from the relabeling classes
      new_label = random.choice(relabeling_classes)
      # Check if the detection label is the specified class
      if detection.label == class_name:
        # Assign the new label to the detection
        detection.label = new_label
        # Print the new label
        print(new_label)

    # Save the modified sample
    sample.save()

  # Save the label error set
  label_error_set.save()

  # Print the number of images with label errors
  print("number of images with label errors = ", len(label_error_set))

  # Concatenate the label error set and error-free set
  concat_data_set = label_error_set.concat(error_free_set)
  # Shuffle the concatenated dataset
  shuffled_data = concat_data_set.shuffle()

  # Check if there are two relabeling classes and insert the specified class at the beginning
  if (len(relabeling_classes)==2):
    relabeling_classes.insert(0, class_name)

  # Return a clone of the shuffled dataset
  return shuffled_data.clone()


In [ ]:
def generateDataset(dataset, class_name, size, resplit_percent):
  train_dataset, test_dataset = clone_and_split_test(dataset, class_name)
  #merged_dataset = merge_dataset(train_dataset, class_name)
  resized_dataset = resize_dataset(train_dataset, size)
  label_error_dataset = add_label_error(resized_dataset, error_rate, class_name)
  train_set, val_set = resplit(label_error_dataset, resplit_percent)


  return train_set, val_set, test_dataset

In [ ]:
class_name = 'cat'
max_samples = 4000
dataset = dwonload_dataset(class_name, max_samples)

# prepare test and valiation data

In [ ]:
exp_name = "E2D30"

size = 30                               # will shrink the size by a percentage if 90 then the new size will be 90% of the orginal
resplit_precent = 80                    # will assign the training set using the precentage given.
error_rate = 0                          # will add error rate according to the precentage given.


train_data, val_data, test_data = generateDataset(dataset, class_name, size, resplit_precent)





In [ ]:
#session = fo.launch_app(val_data)

In [ ]:
print("length of training set = ",len(train_data))
print("length of validation set = ",len(val_data))
print("length of test set = ",len(test_data))
print("********************************************")
i = 0
for sample in val_data:
  i += 1
  #print(i , '- ', sample.ground_truth.detections, ' - ', sample.tags)
  print(i , '- ', sample.ground_truth.detections[0].label, ' - ', sample.tags)
print("********************************************")
print("length of training set = ",len(train_data))
print("length of validation set = ",len(val_data))

In [ ]:
#!rm -rf /content/darknet

In [ ]:
#!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
!pwd

In [ ]:
%cd /content/darknet
%rm -rf data/
%mkdir data/
%cd data
%mkdir labels/
%cd /content/darknet
%rm -rf cfg/
%mkdir cfg

In [ ]:
export_dir = "/content/darknet/data"
data_path = "/content/darknet/data/obj"
labels_path = "/content/darknet/data/obj"
images_path = "/content/darknet/train.txt"
label_field = "ground_truth"  # for example

# The dataset or view to export
dataset_or_view = train_data

# Export the dataset
dataset_or_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv4Dataset,
    label_field=label_field,
    classes= labeling_classes,
    data_path = data_path,
    labels_path = labels_path,
    images_path = images_path,
)
%cd /content/darknet
!mv train.txt data/train.txt

In [ ]:
export_dir = "/content/darknet/data"
data_path = "/content/darknet/data/obj"
labels_path = "/content/darknet/data/obj"
images_path = "/content/darknet/val.txt"
label_field = "ground_truth"  # for example

# The dataset or view to export
dataset_or_view = val_data

# Export the dataset
dataset_or_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv4Dataset,
    label_field=label_field,
    classes= labeling_classes,
    data_path = data_path,
    labels_path = labels_path,
    images_path = images_path,
)
%cd /content/darknet
!mv val.txt data/val.txt

In [ ]:
!cp /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/obj.data /content/darknet/data
#!cp /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/obj.names /content/darknet/build/darknet/x64/data
!cp -r /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/cfg /content/darknet/

In [ ]:
# will show the folder size
!du -sh /content/darknet/data
!du -sh /content/data/{exp_name}
!du -sh /content/terminal/{exp_name}.zip

In [ ]:
%cd data/

In [ ]:
#%cd data/
!sed -i 's@^backup = /mimer/NOBACKUP/groups/naiss2023-22-457/.*@backup = /mimer/NOBACKUP/groups/naiss2023-22-457/Experiments_results/{exp_name}/weights@' obj.data
#!sed -i 's@classes = @classes = 3@' obj.data
!sed -i 's/^classes = .*/classes = 3/' obj.data

In [ ]:
!cat /content/darknet/data/obj.data

In [ ]:
!mkdir -p /content/terminal
!mkdir -p /content/data/{exp_name}/data
!mkdir -p /content/data/{exp_name}/cfg
!mkdir -p /content/data/{exp_name}/weights
!cp -r /content/darknet/data /content/data/{exp_name}
!cp -r /content/darknet/cfg /content/data/{exp_name}
#!mv /content/data/{exp_name}.zip /content/terminal/{exp_name}
#!cp -r /content/terminal/{exp_name}.zip /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/

In [ ]:
!pwd
%cd /content/data/
!zip -r /content/terminal/{exp_name}.zip ./{exp_name}/*
%cd /content/darknet/data

In [ ]:
!cp -r /content/terminal/{exp_name}.zip /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/

In [ ]:
#!zip -r /content/terminal/{exp_name}.zip /content/data/{exp_name}/*

In [ ]:
#!rm -rf /content/data/{exp_name}
#!rm -rf /content/terminal/{exp_name}.zip

#PREPARE TEST DATA
the test data is made of one class "cat" an consist of 3000 images.

In [ ]:
!rm -rf /content/darknet/testData
!rm -rf /content/darknet/cfg
!rm -rf /content/data/testData
!rm -rf /content/terminal/testData.zip
!mkdir -p /content/darknet/testData/labels/
!mkdir -p /content/darknet/cfg

In [ ]:
%cd /content/

In [ ]:

export_dir = "/content/darknet/testData"
data_path = "/content/darknet/testData/obj"
labels_path = "/content/darknet/testData/obj"
images_path = "/content/darknet/test.txt"
label_field = "ground_truth"  # for example

# The dataset or view to export
dataset_or_view = test_data

# Export the dataset
dataset_or_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv4Dataset,
    label_field=label_field,
    classes= labeling_classes,
    data_path = data_path,
    labels_path = labels_path,
    images_path = images_path,
)

!mv /content/darknet/test.txt /content/darknet/testData/test.txt

In [ ]:
!cp /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/obj.data /content/darknet/testData/
#!cp /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/obj.names /content/darknet/build/darknet/x64/data
!cp /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/yolo-obj.cfg /content/darknet/cfg
!cp /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/config/yolo-obj-test.cfg /content/darknet/cfg

In [ ]:
# will show the folder size
!du -sh /content/darknet/testData
!du -sh /content/data/testData
!du -sh /content/terminal/testData.zip

In [ ]:
%cd /content/darknet/testData/

In [ ]:
#%cd data/
exp_name = "testData"
!sed -i 's/^classes = .*/classes = 3/' obj.data
!sed -i 's@^train  =.*@train  = testData/test.txt@' obj.data
!sed -i 's@^valid  =.*@valid  = testData/test.txt@' obj.data
!sed -i 's@^names = .*@names = testData/obj.names@' obj.data
!sed -i 's@^backup = /mimer/NOBACKUP/groups/naiss2023-22-457/.*@backup = /mimer/NOBACKUP/groups/naiss2023-22-457/Experiments_results/{exp_name}/weights@' obj.data
#!sed -i 's@classes = @classes = 3@' obj.data


In [ ]:
!cat /content/darknet/testData/obj.data

In [ ]:
!mkdir -p /content/terminal
!mkdir -p /content/data/{exp_name}/{exp_name}
!mkdir -p /content/data/{exp_name}/cfg
!mkdir -p /content/data/{exp_name}/weights
!cp -r /content/darknet/testData /content/data/{exp_name}
!cp -r /content/darknet/cfg /content/data/{exp_name}
#!mv /content/data/{exp_name}.zip /content/terminal/{exp_name}
#!cp -r /content/terminal/{exp_name}.zip /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/

In [ ]:
!pwd
%cd /content/data/
!zip -r /content/terminal/{exp_name}.zip ./{exp_name}/*
%cd /content/darknet/data

In [ ]:
!cp -r /content/terminal/{exp_name}.zip /content/gdrive/MyDrive/Colab_Notebooks/Experiments_results/

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version

!nvidia-smi
# We need to install the correct cuDNN according to this output

# 7. Download pre-trained weights for the convolutional layers and put to the directory build\darknet\x64

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

#Download pre-trained YOLOv4 weights
YOLOv4 has been trained already on the coco dataset which has 80 classes that it can predict. We will grab these pretrained weights so that we can run YOLOv4 on these pretrained classes and get detections.

In [ ]:
%cd /content/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
%cd /content/darknet

# Train Your Custom Object Detector!
The time has finally come! You have made it to the moment of truth! You are now ready to train your custom YOLOv4 object detector on whatever crazy classes you have decided on. So run the following command. (-dont_show flag stops chart from popping up since Colab Notebook can't open images on the spot, -map flag overlays mean average precision on chart to see how accuracy of your model is, only add map flag if you have a validation dataset)
```
!./darknet detector train <path to obj.data> <path to custom config> yolov4.conv.137 -dont_show -map
```
**TIP:** This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working");
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click()
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

In [ ]:
!pwd
%cd /content/darknet
!pwd

In [ ]:
!chmod +x ./darknet

In [ ]:
!./darknet detector train data/obj.data cfg/yolo-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train /content/darknet/build/darknet/x64/data/obj.data /content/darknet/build/darknet/x64/cfg/yolo-obj.cfg /content/darknet/build/darknet/x64/yolov4.conv.137 -dont_show -map

In [ ]:
data1.add_samples(test_data)

In [ ]:
i= 0
for sample in data1.limit(20):
  i += 1
  print(i , '- ', '', '-', sample.ground_truth.detections)

In [ ]:
print(fo.list_datasets())             #should be three dataset

In [ ]:
# will show the folder size
!du -sh /content/data

In [ ]:
!rm -rf /content/darknet/data/data